In [1]:
%%time
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\programdata\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.
Wall time: 7.95 s


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

import glob

import argparse
import logging
import os

from pathlib import Path

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201107


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
#心斎橋店ファイルで味見

In [8]:
data_dir = Path(r"C:\Users\crie072\Downloads")

#input
input_new = "買上明細VIEW心斎橋"
import codecs

In [31]:
df = pd.read_csv(data_dir /"買上明細VIEW心斎橋 (1).txt",encoding='utf-16')
df

,kojin_id,tr_ymd,pos_no,tr_no,pay_shu,toitukozasbt,hinban_cd,hnban_mei,Sum(det_uri_kin) (用),購買月
0,3,20190710,9501,234,1,117,810,和洋酒,2700,1907
1,35,20191207,2056,1738,7,117,551,紳士服飾 テナント,43000,1912
2,38,20190311,1111,2586,9,117,530,ワイシャツ,15000,1903
3,38,20190423,1111,3310,9,117,530,ワイシャツ,10000,1904
4,38,20190520,1150,5774,9,117,810,和洋酒,6964,1905
...,...,...,...,...,...,...,...,...,...,...
2751752,500252933,20200212,136,6407,1,116,881,フーズ テナント,1782,2002
2751753,500252933,20200213,124,6873,1,116,881,フーズ テナント,565,2002
2751754,500252933,20200216,127,7459,0,116,881,フーズ テナント,170,2002
2751755,500252933,20200218,101,646,1,116,881,フーズ テナント,4000,2002


In [32]:
df_tmp = df[:10]
df_tmp

,kojin_id,tr_ymd,pos_no,tr_no,pay_shu,toitukozasbt,hinban_cd,hnban_mei,Sum(det_uri_kin) (用),購買月
0,3,20190710,9501,234,1,117,810,和洋酒,2700,1907
1,35,20191207,2056,1738,7,117,551,紳士服飾 テナント,43000,1912
2,38,20190311,1111,2586,9,117,530,ワイシャツ,15000,1903
3,38,20190423,1111,3310,9,117,530,ワイシャツ,10000,1904
4,38,20190520,1150,5774,9,117,810,和洋酒,6964,1905
5,38,20190520,8933,2810,9,117,682,レストラン３,1800,1905
6,38,20190520,1111,9534,9,117,506,ビジネス・フォーマルウェア,16000,1905
7,38,20190520,3113,2129,9,117,721,食器・調理用品,5000,1905
8,38,20190520,1111,9531,9,117,530,ワイシャツ,15000,1905
9,38,20190726,3113,3472,9,117,721,食器・調理用品,5000,1907


In [31]:
#CSV出力
df_tmp.to_csv(data_dir /f'{input_new}_味見.csv', header=True ,index=None, encoding='shift=jis')

In [33]:
print(df.dtypes)

kojin_id                 int64
tr_ymd                   int64
pos_no                   int64
tr_no                    int64
pay_shu                  int64
toitukozasbt             int64
hinban_cd                int64
hnban_mei               object
Sum(det_uri_kin) (用)     int64
購買月                      int64
dtype: object


In [34]:
print(df['toitukozasbt'].unique())

[ 117  120  111  116 1011  118  101  119 1001  115 1015]


統一口座種別<br>
101	　旧　大丸外商カード<br>
111	　大丸友の会カード<br>
114	　Webサークル<br>
115	　ブライダルサークル<br>
116	　大丸松坂屋カード<br>
117	　大丸松坂屋ポイントカード（旧Dカード、Mカード含む）<br>
118	　大丸松坂屋カードGOLD<br>
119	　うふふガールズカード<br>
120	　新お得意様GCカード<br>
1001　旧　松坂屋外商カード<br>
1011　松坂屋友の会カード<br>

In [35]:
print(df['pay_shu'].unique())

[1 7 9 0 2 4 5 8]


支払種別<br>
0 引き落としの支払でないもの<br>
1 一括払い<br>
2 分割払い（3回以上）<br>
4 リボ払い<br>
5 ボーナス一括払い<br>
7 分割払い（2回）<br>
8 ??<br>
9 全額入金<br>

In [36]:
df['Sum(det_uri_kin) (用)'].describe()/1000

count     2751.757000
mean        13.038690
std        149.286964
min     -35980.000000
25%          0.800000
50%          1.750000
75%          5.000000
max      81818.181000
Name: Sum(det_uri_kin) (用), dtype: float64

In [18]:
df.groupby(['購買月'])['pos_no'].count()

購買月
1903    212953
1904    196471
1905    193025
1906    226894
1907    230960
1908    207941
1909    268931
1910    268545
1911    256605
1912    290823
2001    207433
2002    191176
Name: pos_no, dtype: int64

In [37]:
print(df['hnban_mei'].unique())

['和洋酒' '紳士服飾\u3000テナント' 'ワイシャツ' 'レストラン３' 'ビジネス・フォーマルウェア' '食器・調理用品'
 'フーズ\u3000テナント' 'レストラン２（新館売仕）' '精肉・ハム' 'グロッサリー' '惣菜' '和菓子' '海産物' '洋菓子'
 '化粧品' 'リビング雑貨' '青果物' '子供服' 'インターナショナルブティック' 'エステティック' 'デリカ' '宝飾品'
 'ミセス・毛皮' 'メガネ' 'ベーカリー' 'トラベルショップ' '結納用品' '婦人服\u3000テナント' '家庭電器'
 '住文化用品\u3000テナント' '外販食品' '美味リクエスト便' '肌着・靴下・ナイティ' '婦人洋品' 'ネクタイ洋品' 'きもの'
 '婦人肌着' 'ＹＡキャラクター' 'ゴルフウェア' '婦人靴' 'ライフスタイル雑貨' 'ＮＭ１プレタポルテ' '紳士靴' 'トロージャン'
 'イージーオーダー' '婦人雑貨子供服\u3000テナント' nan '寝具' '紳士服飾催（催事運営）' 'オーセンティックプレタ'
 'カジュアルウェア' '時計' '絵画' 'その他催' 'フリーチョイス' '婦人服催（催事運営）' '文化催事（催事運営）' 'アクセサリー'
 '特選ブティック' 'セレクトファッション' '婦人雑貨特選ショップ' 'フリースペース' 'インターナショナルブティック周辺店舗'
 '総合服飾\u3000テナント' 'エステ・トラベル' 'ファッショングッズ' 'サービス\u3000テナント\u3000フドウサンジギョウブ'
 'フリーチョイスカタログ' '食品ギフト券' '婦人雑貨催（催事運営）' '住文催（催事運営）' '家具・敷物'
 '美術・呉服・宝飾\u3000テナント' '六甲インテリア館' 'サービス\u3000テナント' '特選ブランドカード提携'
 'カード提携周辺店舗' '特選大口カード提携' '不動産事業部物販テナント' '酒類券' 'ＷＥＢ計上（リビング他）' '学生服'
 '図書券（大口）' 'カバン・旅行用品' 'ＷＥＢご当地自慢' 'クリーニング' '婦人洋品名品店' '調整品番' '周辺大口' '挨拶状'
 '婦人雑貨子供服\u3000テナント\u3000不動産事業部' 'ＳＰグッ

In [5]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_Spotfile百貨店購買データ")

In [5]:
path = r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_Spotfile百貨店購買データ" # use your path
all_files = glob.glob(path + "/*.txt")

In [6]:
len(path)

65

In [6]:
%%time
path = r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_Spotfile百貨店購買データ" # use your path
all_files = glob.glob(path + "/*.txt")

li = []
 
for filename in all_files:
    df = pd.read_csv(filename
                     ,encoding='utf-16')
    df['shop_name'] = filename[74:-4]
    li.append(df)
df_all = pd.concat(li, axis=0, ignore_index=True)
df_all.head()

Wall time: 2min 23s


,kojin_id,tr_ymd,pos_no,tr_no,pay_shu,toitukozasbt,hinban_cd,hnban_mei,Sum(det_uri_kin) (用),購買月,shop_name
0,37,20191101,4025,5617,0,116,831,ベーカリー,1080,1911,京都
1,81,20190430,3014,534,9,117,830,惣菜,1830,1904,京都
2,172,20190315,4023,555,9,120,831,ベーカリー,280,1903,京都
3,172,20190315,4164,2274,9,120,821,精肉・ハム,2400,1903,京都
4,172,20190315,6110,1039,9,120,821,精肉・ハム,471,1903,京都


In [7]:
df_all.tail()

,kojin_id,tr_ymd,pos_no,tr_no,pay_shu,toitukozasbt,hinban_cd,hnban_mei,Sum(det_uri_kin) (用),購買月,shop_name
49034420,500272524,20190803,5004,808,9,118,822,青果物,554,1908,神戸
49034421,500272524,20190803,6157,4599,9,118,830,惣菜,2507,1908,神戸
49034422,500272524,20190803,5004,808,9,118,811,デリカ,1150,1908,神戸
49034423,500272524,20190803,6132,9365,9,118,830,惣菜,130,1908,神戸
49034424,500272524,20190803,6198,4715,9,118,800,洋菓子,2600,1908,神戸


In [8]:
df_all.shape[0]

49034425

In [9]:
df_all['kojin_id'].nunique()

2483863

In [ ]:
#全体集計

In [10]:
df_count = df_all.groupby(['shop_name'])['kojin_id'].count()
df_count

shop_name
京都     9353057
名古屋    9151087
心斎橋    2751757
札幌     7974137
東京     5963290
梅田     6503769
神戸     7337328
Name: kojin_id, dtype: int64

In [11]:
df_uu = df_all.groupby('shop_name')['kojin_id'].nunique()
df_uu

shop_name
京都     403987
名古屋    371105
心斎橋    306208
札幌     435962
東京     588531
梅田     661554
神戸     437981
Name: kojin_id, dtype: int64

In [13]:
df_sum =df_all.groupby(['shop_name'])['Sum(det_uri_kin) (用)'].sum()
df_sum

shop_name
京都     44688094052
名古屋    78049393024
心斎橋    35879306165
札幌     37073444967
東京     34831284311
梅田     35354020643
神戸     56669816143
Name: Sum(det_uri_kin) (用), dtype: int64

In [14]:
df_merge = pd.merge(df_count,df_uu,on = ['shop_name'], how = 'left')
df_merge = pd.merge(df_merge,df_sum, on = ['shop_name'], how = 'left')

In [15]:
df_merge

,kojin_id_x,kojin_id_y,Sum(det_uri_kin) (用)
shop_name,,,
京都,9353057,403987,44688094052
名古屋,9151087,371105,78049393024
心斎橋,2751757,306208,35879306165
札幌,7974137,435962,37073444967
東京,5963290,588531,34831284311
梅田,6503769,661554,35354020643
神戸,7337328,437981,56669816143


In [16]:
#CSV出力
df_merge.to_csv(data_dir /r"Output"/'spotfile_店別全体集計.csv', header=True ,encoding='shift=jis')

In [ ]:
#店別、売り場別、カード種類別

In [17]:
df_count = df_all.groupby(['shop_name','hnban_mei','toitukozasbt'])['kojin_id'].count()
df_count.head()

shop_name  hnban_mei  toitukozasbt
京都         きもの        101              173
                      111              569
                      115                4
                      116             1267
                      117             5193
Name: kojin_id, dtype: int64

In [18]:
df_sum =df_all.groupby(['shop_name','hnban_mei','toitukozasbt'])['Sum(det_uri_kin) (用)'].sum()
df_sum.head()

shop_name  hnban_mei  toitukozasbt
京都         きもの        101              15218497
                      111               7365387
                      115                 24540
                      116              22003205
                      117             125701879
Name: Sum(det_uri_kin) (用), dtype: int64

In [19]:
df_merge = pd.merge(df_count,df_sum,on = ['shop_name','hnban_mei','toitukozasbt'], how = 'left')
df_merge.head()

kojin_id  Sum(det_uri_kin) (用)
shop_name hnban_mei toitukozasbt                                
京都        きもの       101                173              15218497
                    111                569               7365387
                    115                  4                 24540
                    116               1267              22003205
                    117               5193             125701879

In [20]:
#CSV出力
df_merge.to_csv(data_dir /r"Output"/'spotfile_店別売り場別集計.csv', header=True ,encoding='shift=jis')

In [ ]:
#店別、カード別　件数、UU、金額

In [21]:
df_count = df_all.groupby(['shop_name','toitukozasbt'])['kojin_id'].count()
df_count.head()

shop_name  toitukozasbt
京都         101               78168
           111              637737
           115                 562
           116             1658394
           117             4700001
Name: kojin_id, dtype: int64

In [22]:
df_uu = df_all.groupby(['shop_name','toitukozasbt'])['kojin_id'].nunique()
df_uu.head()

shop_name  toitukozasbt
京都         101               2616
           111              38344
           115                485
           116             138770
           117             198088
Name: kojin_id, dtype: int64

In [23]:
df_sum =df_all.groupby(['shop_name','toitukozasbt'])['Sum(det_uri_kin) (用)'].sum()
df_sum.head()

shop_name  toitukozasbt
京都         101              1069448355
           111              1856143110
           115                73781651
           116              5059372299
           117             18753552826
Name: Sum(det_uri_kin) (用), dtype: int64

In [24]:
df_merge = pd.merge(df_count,df_uu,on = ['shop_name','toitukozasbt'], how = 'left')
df_merge = pd.merge(df_merge,df_sum, on = ['shop_name','toitukozasbt'], how = 'left')

In [25]:
df_merge

kojin_id_x  kojin_id_y  Sum(det_uri_kin) (用)
shop_name toitukozasbt                                              
京都        101                78168        2616            1069448355
          111               637737       38344            1856143110
          115                  562         485              73781651
          116              1658394      138770            5059372299
          117              4700001      198088           18753552826
...                            ...         ...                   ...
神戸        118               792651       29052            5844382659
          119                17678        1837              95094001
          120              1335397       48419           18924591419
          1001                 149          64               4958682
          1011                1974         562               8675803

[74 rows x 3 columns]

In [26]:
#CSV出力
df_merge.to_csv(data_dir /r"Output"/'spotfile_店別カード別集計.csv', header=True ,encoding='shift=jis')

In [ ]:
#店別、支払い手段別　件数、UU、金額

In [27]:
df_count = df_all.groupby(['shop_name','pay_shu'])['kojin_id'].count()
df_count.head()

shop_name  pay_shu
京都         0          1952960
           1          3374925
           2            11003
           4             1547
           5             8000
Name: kojin_id, dtype: int64

In [28]:
df_uu = df_all.groupby(['shop_name','pay_shu'])['kojin_id'].nunique()
df_uu.head()

shop_name  pay_shu
京都         0          156131
           1          240343
           2            5175
           4             508
           5            3559
Name: kojin_id, dtype: int64

In [29]:
df_sum =df_all.groupby(['shop_name','pay_shu'])['Sum(det_uri_kin) (用)'].sum()
df_sum.head()

shop_name  pay_shu
京都         0           4634443324
           1          26771707049
           2            906192430
           4             42939067
           5            404767173
Name: Sum(det_uri_kin) (用), dtype: int64

In [30]:
df_merge = pd.merge(df_count,df_uu,on = ['shop_name','pay_shu'], how = 'left')
df_merge = pd.merge(df_merge,df_sum, on = ['shop_name','pay_shu'], how = 'left')

In [29]:
df_merge

kojin_id_x  kojin_id_y  Sum(det_uri_kin) (用)
shop_name pay_shu                                              
京都        0           1952960      156131            4634443324
          1           3374925      240343           26771707049
          2             11003        5175             906192430
          4              1547         508              42939067
          5              8000        3559             404767173
          7             40631       14128            1414869616
          9           3963991      204998           10513175393
心斎橋       0            583424      104635            3061434483
          1           1198120      183969           24856124565
          2              4080        2465             526160703
          4               543         242              27431519
          5              4812        2475             362654217
          7             20772        9178            1130160967
          8                 1           1                -30800
          9            940005      117316            5915370511
東京        0           2200543      278341            5325119427
          1           2511451      364492           23080527401
          2              5181        2920             407034082
          4               892         456              34301654
          5              6937        3580             364904151
          7             34470       13795            1259236024
          9           1203816      154198            4360161572
神戸        0           1266347      151902            4499971205
          1           2782157      282580           36067977671
          2             11588        6207            1058313555
          3                 1           1                755000
          4              1387         558              47637627
          5             13208        6073             734362104
          7             61113       21380            2410994626
          8                 1           1               -125000
          9           3201526      223770           11849929355

In [31]:
#CSV出力
df_merge.to_csv(data_dir /r"Output"/'spotfile_店別支払い手段別集計.csv', header=True ,encoding='shift=jis')

In [32]:
df_all = df_all.rename(columns={'Sum(det_uri_kin) (用)':'uri_kin','購買月':'tr_yymm'})

In [38]:
df_all.head()

,kojin_id,tr_ymd,pos_no,tr_no,pay_shu,toitukozasbt,hinban_cd,hnban_mei,uri_kin,tr_yymm,shop_name
0,37,20191101,4025,5617,0,116,831,ベーカリー,1080,1911,京都
1,81,20190430,3014,534,9,117,830,惣菜,1830,1904,京都
2,172,20190315,4023,555,9,120,831,ベーカリー,280,1903,京都
3,172,20190315,4164,2274,9,120,821,精肉・ハム,2400,1903,京都
4,172,20190315,6110,1039,9,120,821,精肉・ハム,471,1903,京都


In [39]:
df_all.shape[0]

25405432

In [10]:
%%time
#集計
q = """
        SELECT
        shop_name
        ,tr_yymm
        ,toitukozasbt
        ,pay_shu
        ,hinban_cd
        ,hnban_mei
        ,count(kojin_id) as no_of_tran
        ,count(distinct kojin_id) as uu
        ,sum(uri_kin) as sum_uri_kin
        from 
            df_all
        group by 
            1,2,3,4,5,6
        """
df_shukei = sqldf(q, locals())
df_shukei

,shop_name,tr_yymm,toitukozasbt,pay_shu,hinban_cd,hnban_mei,no_of_tran,uu,sum_uri_kin
0,心斎橋,1903,101,1,250,結納用品,6,6,563500
1,心斎橋,1903,101,1,260,フリーチョイス,3,3,57200
2,心斎橋,1903,101,1,262,クリーニング,11,2,125381
3,心斎橋,1903,101,1,268,挨拶状,1,1,8500
4,心斎橋,1903,101,1,300,インターナショナルブティック,78,46,20095067
...,...,...,...,...,...,...,...,...,...
25019,東京,2002,1011,0,840,レストラン,64,56,341098
25020,東京,2002,1011,0,841,各階喫茶・軽食,45,41,86249
25021,東京,2002,1011,0,872,時計,3,3,42000
25022,東京,2002,1011,1,406,ハンドバッグ,1,1,35000


In [11]:
#CSV出力
df_shukei.to_csv(data_dir /r"Output"/'spotfile_集計.csv', header=True ,index=None, encoding='shift=jis')

In [ ]:
%%time
#集計
q = """
        SELECT
        shop_name
        ,toitukozasbt
        ,hnban_mei
        ,count(kojin_id) as no_of_tran
        ,count(distinct kojin_id) as uu
        ,sum(uri_kin) as sum_uri_kin
        from 
            df_all
        group by 
            1,2,3
        """
df_shukei2 = sqldf(q, locals())
df_shukei2

In [ ]:
#CSV出力
df_shukei.to_csv(data_dir /r"Output"/'spotfile_集計2.csv', header=True ,index=None, encoding='shift=jis')